In [1]:
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from typing import TypedDict


In [2]:
load_dotenv()

True

In [3]:
model=ChatOpenAI()


In [4]:
class EmailState(TypedDict):
    Email: str
    intent: str
    reply: str

In [8]:
intent_prompt=PromptTemplate(template="""
                      classify the intent of the following email.\n
                      Possible intents: Complaint, refund, general \n\n
                      Email:\n{email}\n\n
                      Return only the intent""",
                      input_variables=['email'])

In [6]:
reply_prompt=PromptTemplate(template="""
                            Write a professional reply for the following email.\n
                            Intent:{intent}\n
                            Email:{email}""",
                            input_variables=["intent","email"])

In [9]:
intent_chain=intent_prompt | model
reply_chain=reply_prompt | model

In [11]:
def detect_intent(state: EmailState)->EmailState:
    response=intent_chain.invoke({
        'email':state['Email']
    })
    state['intent']=response.content.strip()
    return state

In [12]:
def generate_email(state: EmailState)->EmailState:
    response=reply_chain.invoke({
        'email':state['Email'],
        'intent':state['intent']
    })
    state['reply']=response.content
    return state

In [13]:
#define graph
graph=StateGraph(EmailState)

# Add nodes
graph.add_node('detect_intent',detect_intent)
graph.add_node('generate_email',generate_email)
# Add edges
graph.add_edge(START, 'detect_intent')
graph.add_edge('detect_intent','generate_email')
graph.add_edge('generate_email', END)
# Compile
workflow=graph.compile()

In [15]:
final_state=workflow.invoke({
    'Email':'I was charged twice for my subscription. Please fix this'
})
print(final_state['intent'])
print(final_state['reply'])

Complaint
Dear [Customer],

Thank you for reaching out and bringing this issue to our attention. We apologize for the inconvenience and the double charge on your subscription. We will investigate this matter further and ensure that the duplicate charge is refunded to you promptly. Please provide us with your account details or transaction information so that we can resolve this issue as quickly as possible.

We appreciate your understanding and patience as we work to rectify this situation.

Thank you for your continued support.

Sincerely,
[Your Name]
[Company Name]
